<a href="https://colab.research.google.com/github/codebybishwa/Amazon-ML-Challenge/blob/main/Amazon_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load your train and test datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')


# Randomly select 20 samples from the filtered data
dumy_train = train_df.sample(n=10000, random_state=42)
dumy_test = test_df.sample(n=10000, random_state=42)

# Save the new datasets to CSV files
dumy_train.to_csv('dumy_train.csv', index=False)
dumy_test.to_csv('dumy_test.csv', index=False)


In [ ]:
dummy_train = pd.read_csv('dumy_train.csv')
dummy_test = pd.read_csv('dumy_test.csv')

In [ ]:
print(dummy_train.head())
print(dummy_test.head())


                                          image_link  group_id  entity_name  \
0  https://m.media-amazon.com/images/I/81nEfmLyyM...    630390  item_weight   
1  https://m.media-amazon.com/images/I/51gz4v5BxT...    751532        width   
2  https://m.media-amazon.com/images/I/61D2A-LxNn...    478233       height   
3  https://m.media-amazon.com/images/I/51INxCAF0T...    943839        depth   
4  https://m.media-amazon.com/images/I/81-zmM+W8B...    487566  item_weight   

   entity_value  
0  1.0 kilogram  
1     78.0 inch  
2     10.1 inch  
3     2.88 inch  
4     1.8 pound  
    index                                         image_link  group_id  \
0   81578  https://m.media-amazon.com/images/I/610apmjypy...    311997   
1  120463  https://m.media-amazon.com/images/I/71WGncjrqQ...    355666   
2   89428  https://m.media-amazon.com/images/I/6158kNWFcS...    407808   
3   53428  https://m.media-amazon.com/images/I/51Zj1+VgNT...    267482   
4   41007  https://m.media-amazon.com/images/I/

In [ ]:
!sudo apt-get install tesseract-ocr
!pip install pytesseract Pillow pandas

import pytesseract
from PIL import Image
import pandas as pd
import requests
from io import BytesIO
# Import the required class
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm

# Load the dataset
  # Make sure the file name is correct


def extract_text_from_image(url):
    try:
        response = requests.get(url, timeout=10)
        img = Image.open(BytesIO(response.content))
        text = pytesseract.image_to_string(img)
        return text.strip()
    except Exception as e:
        print(f"Error processing image {url}: {e}")
        return ""

def process_images(urls):
    results = []
    with ProcessPoolExecutor() as executor:
        futures = [executor.submit(extract_text_from_image, url) for url in urls]
        for future in tqdm(as_completed(futures), total=len(urls)):
            results.append(future.result())
    return results

# Assuming dummy_train is your DataFrame
urls = dummy_train['image_link'].tolist()

# Process images in parallel
extracted_texts = process_images(urls)

# Add extracted text to the DataFrame
dummy_train['extracted_text'] = extracted_texts

print("Updated DataFrame with Extracted Text:")
print(dummy_train.head())
# Extract text for each image URL
dummy_train['extracted_text'] = dummy_train['image_link'].apply(extract_text_from_image)

In [ ]:
import re
import pandas as pd

# Updated regular expressions for each entity type
unit_patterns = {
    'item_weight': r'\b(\d+\.?\d*)\s*(kg|kgf|g|gm|lb|pound|oz|ounce|MG|milligram|ton|T|milligrams|POUND|KILOGRAM|GRAM|GM|LB|OUNCE|MILLIGRAM|TON|T)\b',
    'temperature': r'\b(\d+\.?\d*)\s*(°C|°F|degrees|celsius|fahrenheit|K|kelvin|DEGREES|CELSIUS|FAHRENHEIT|KELVIN|DEG|CEL|FAH|F|C)\b',
    'volume': r'\b(\d+\.?\d*)\s*(ml|milliliter|milliliters|l|liter|liters|ML|L|LITERS|MILLILITER|GALLON|GALLONS|CC|cubic cm|cubic centimeter|m³|M³|cubic meter|m^3|M^3)\b',
    'length': r'\b(\d+\.?\d*)\s*(cm|millimeter|millimeters|mm|inch|in|inch|ft|feet|m|meter|meter|CM|MM|IN|INCH|FT|FEET|M|METER|YARD|YD|mile|MI|NAUTICAL MILE|NM|nanometer|MICROMETER|µm|millimeter|MM²)\b',
    'voltage': r'\b(\d+\.?\d*)\s*(V|volt|volts|mV|millivolt|kV|kilovolt|V|VOLTS|MILLIVOLT|KILOVOLT)\b',
    'power': r'\b(\d+\.?\d*)\s*(W|watt|watts|kW|kilowatt|mW|milliwatt|MW|megawatt|W|WATTS|KILOWATT|MILLIWATT|MEGAWATT)\b',
    'dimension': r'\b(\d+\.?\d*)\s*(x)\s*(\d+\.?\d*)\s*(cm|millimeter|mm|inch|in|ft|feet|m|meter|CM|MM|IN|INCH|FT|FEET|M|METER|YARD|YD|km|kilometer|MI|mile|nautical mile|NM|nanometer|MICROMETER|µm)\b',
    'area': r'\b(\d+\.?\d*)\s*(m²|m^2|square meter|ft²|square foot|in²|square inch|ac|acre|hectare|ha|cm²|square centimeter|mm²|square millimeter)\b',
    'speed': r'\b(\d+\.?\d*)\s*(km/h|kmph|kph|mph|mile/h|m/s|meter/s|ft/s|foot/s|kmph|KPH|M/S|MILE/H|KM/H|M/S)\b',
    'density': r'\b(\d+\.?\d*)\s*(kg/m³|kilogram/m³|g/cm³|gram/cm³|lb/ft³|pound/ft³|kg/m^3|g/cm^3|lb/ft^3|kg/m³|g/cm³)\b',
    'time': r'\b(\d+\.?\d*)\s*(s|sec|second|min|minute|h|hour|hr|day|wk|week|mo|month|yr|year|sec|minute|hour|day|week|month|year|S|SEC|SECOND|MIN|MINUTE|H|HR|DAY|WK|WEEK|MO|MONTH|YR|YEAR)\b',
    'energy': r'\b(\d+\.?\d*)\s*(J|joule|joules|kJ|kilojoule|cal|calorie|kcal|kilocalorie|Wh|watt-hour|kWh|kilowatt-hour|MWh|megawatt-hour|J|JOULE|JOULES|KJ|KILOJOULE|CAL|CALORIE|KCAL|KILOCALORIE|WH|WATT-HOUR|KWH|KILOWATT-HOUR|MWH|MEGAWATT-HOUR)\b',
    'pressure': r'\b(\d+\.?\d*)\s*(Pa|pascal|kPa|kilopascal|bar|psi|pound per square inch|atm|atmosphere|BAR|ATM|PAS|KPA|KILOPASCAL|PSI|PA|PRESSURE|ATMOSPHERE)\b',
    'frequency': r'\b(\d+\.?\d*)\s*(Hz|hertz|kHz|kilohertz|MHz|megahertz|GHz|gigahertz|Hz|HERTZ|KHZ|KILOHERTZ|MHZ|MEGAHERTZ|GHZ|GIGAHERTZ)\b',
    'data_storage': r'\b(\d+\.?\d*)\s*(B|byte|bytes|KB|kilobyte|MB|megabyte|GB|gigabyte|TB|terabyte|PB|petabyte|EB|exabyte|ZB|zettabyte|YB|yottabyte|BYTES|KILOBYTE|MEGABYTE|GIGABYTE|TERABYTE|PETABYTE|EXABYTE|ZETTABYTE|YOTTABYTE)\b',
    'revolutions': r'\b(\d+\.?\d*)\s*(rpm|revolutions per minute|rev/min|revs/min|REV/MIN|REVOLUTIONS PER MINUTE)\b'
}


# Function to extract values based on entity type
def extract_units_by_entity(row):
    entity = row['entity_name']  # Assuming 'entity_column' holds entity names
    text = row['extracted_text']

    # Check if the entity exists in our unit patterns
    if entity in unit_patterns:
        pattern = unit_patterns[entity]
        matches = re.findall(pattern, text, flags=re.IGNORECASE)

        # If matches found, combine numeric value and unit
        if matches:
            extracted_values = [' '.join(match) for match in matches]
            return ', '.join(extracted_values)

    return None

# Apply the extraction function to each row of the DataFrame
dummy_train['extracted_units'] = dummy_train.apply(extract_units_by_entity, axis=1)

# Filter out rows where no units were extracted
dummy_train_with_units = dummy_train[dummy_train['extracted_units'].notnull()]

# Print or save the result
dummy_train_with_units.to_csv('dummy_train_with_units.csv', index=False)

print("Dataset with extracted units has been saved as 'dummy_train_with_units.csv'.")


Dataset with extracted units has been saved as 'dummy_train_with_units.csv'.


In [ ]:
dummy_train[
    ['extracted_units','entity_name','extracted_text']
]

,extracted_units,entity_name,extracted_text
0,"1 KG, 100 g, 50 mg, 27 mg, 105 g, 64 g",item_weight,ASOSOZERO\n\nBLACK COOKIES FLAVOUR\nFR: Prépar...
1,None,width,
2,None,height,«\n\nx
3,None,depth,/\n\nBOSCH ‘i HSS-Bimetal\n\ngianm |\n27s i\n\...
4,1.8 lb,item_weight,ste Extra Large & Comfortable\n\nfor 4 to 6 pe...
...,...,...,...
995,None,width,170mm/0.6in\n\n \n\n \n\nsonmittin\s '\n\n73mm...
996,None,item_weight,E wiggles.in® Bog 3 wiggles.in’ i)\nLAMB & PEA...
997,None,depth,
998,50 g,item_weight,MN yg\na\n\nNehmen Sie jeden 5. Tag bt er mu\n...


In [ ]:
import pandas as pd
import re


# Dictionary to map common abbreviations to full unit names
# Dictionary to map common abbreviations to full unit names
unit_mapping = {
    # Weight Units
    'g': 'gram', 'G': 'gram', 'gm': 'gram', 'GM': 'gram', 'gram': 'gram', 'GRAM': 'gram',
    'kg': 'kilogram', 'KG': 'kilogram', 'kilogram': 'kilogram', 'KILOGRAM': 'kilogram',
    'lb': 'pound', 'LB': 'pound', 'lbs': 'pound', 'LBS': 'pound', 'pound': 'pound', 'POUND': 'pound',
    'oz': 'ounce', 'OZ': 'ounce', 'ounce': 'ounce', 'OUNCE': 'ounce',
    'mg': 'milligram', 'MG': 'milligram', 'milligram': 'milligram', 'MILLIGRAM': 'milligram',
    'ton': 'ton', 'TON': 'ton', 'tons': 'ton', 'TONS': 'ton',

    # Length Units
    'm': 'meter', 'M': 'meter', 'meter': 'meter', 'METER': 'meter', 'meters': 'meter', 'METERS': 'meter',
    'cm': 'centimeter', 'CM': 'centimeter', 'centimeter': 'centimeter', 'CENTIMETER': 'centimeter',
    'mm': 'millimeter', 'MM': 'millimeter', 'millimeter': 'millimeter', 'MILLIMETER': 'millimeter',
    'km': 'kilometer', 'KM': 'kilometer', 'kilometer': 'kilometer', 'KILOMETER': 'kilometer',
    'in': 'inch', 'IN': 'inch', 'inch': 'inch', 'INCH': 'inch', 'inches': 'inch', 'INCHES': 'inch',
    'ft': 'feet', 'FT': 'feet', 'foot': 'feet', 'FOOT': 'feet', 'feet': 'feet', 'FEET': 'feet',
    'yd': 'yard', 'YD': 'yard', 'yard': 'yard', 'YARD': 'yard',
    'mile': 'mile', 'MILE': 'mile', 'miles': 'mile', 'MILES': 'mile',

    # Volume Units
    'l': 'liter', 'L': 'liter', 'litre': 'liter', 'LITRE': 'liter', 'liters': 'liter', 'LITERS': 'liter',
    'ml': 'milliliter', 'ML': 'milliliter', 'milliliter': 'milliliter', 'MILLILITER': 'milliliter',
    'gallon': 'gallon', 'GALLON': 'gallon', 'gallons': 'gallon', 'GALLONS': 'gallon',
    'fl oz': 'fluid ounce', 'FL OZ': 'fluid ounce', 'fluid ounce': 'fluid ounce', 'FLUID OUNCE': 'fluid ounce',
    'pint': 'pint', 'PINT': 'pint', 'pints': 'pint', 'PINTS': 'pint',
    'quart': 'quart', 'QUART': 'quart', 'quarts': 'quart', 'QUARTS': 'quart',

    # Temperature Units
    'c': 'celsius', 'C': 'celsius', 'celsius': 'celsius', 'CELSIUS': 'celsius', '°C': 'celsius',
    'f': 'fahrenheit', 'F': 'fahrenheit', 'fahrenheit': 'fahrenheit', 'FAHRENHEIT': 'fahrenheit', '°F': 'fahrenheit',
    'kelvin': 'kelvin', 'KELVIN': 'kelvin', 'K': 'kelvin',

    # Voltage Units
    'v': 'volt', 'V': 'volt', 'volt': 'volt', 'VOLT': 'volt', 'volts': 'volt', 'VOLTS': 'volt',
    'kv': 'kilovolt', 'KV': 'kilovolt', 'kilovolt': 'kilovolt', 'KILOVOLT': 'kilovolt',
    'mv': 'millivolt', 'MV': 'millivolt', 'millivolt': 'millivolt', 'MILLIVOLT': 'millivolt',

    # Power Units
    'w': 'watt', 'W': 'watt', 'watt': 'watt', 'WATT': 'watt', 'watts': 'watt', 'WATTS': 'watt',
    'kw': 'kilowatt', 'KW': 'kilowatt', 'kilowatt': 'kilowatt', 'KILOWATT': 'kilowatt',
    'mw': 'megawatt', 'MW': 'megawatt', 'megawatt': 'megawatt', 'MEGAWATT': 'megawatt',

    # Current Units
    'a': 'ampere', 'A': 'ampere', 'amp': 'ampere', 'AMP': 'ampere', 'amps': 'ampere', 'AMPS': 'ampere',
    'ma': 'milliampere', 'MA': 'milliampere', 'milliampere': 'milliampere', 'MILLIAMPERE': 'milliampere',
    'ua': 'microampere', 'UA': 'microampere', 'microampere': 'microampere', 'MICROAMPERE': 'microampere',
    'ka': 'kiloampere', 'KA': 'kiloampere', 'kiloampere': 'kiloampere', 'KILOAMPERE': 'kiloampere',

    # Frequency Units
    'hz': 'hertz', 'HZ': 'hertz', 'hertz': 'hertz', 'HERTZ': 'hertz',
    'khz': 'kilohertz', 'KHZ': 'kilohertz', 'kilohertz': 'kilohertz', 'KILOHERTZ': 'kilohertz',
    'mhz': 'megahertz', 'MHZ': 'megahertz', 'megahertz': 'megahertz', 'MEGAHERTZ': 'megahertz',
    'ghz': 'gigahertz', 'GHZ': 'gigahertz', 'gigahertz': 'gigahertz', 'GIGAHERTZ': 'gigahertz',

    # Pressure Units
    'pa': 'pascal', 'PA': 'pascal', 'pascal': 'pascal', 'PASCAL': 'pascal',
    'kpa': 'kilopascal', 'KPA': 'kilopascal', 'kilopascal': 'kilopascal', 'KILOPASCAL': 'kilopascal',
    'mpa': 'megapascal', 'MPA': 'megapascal', 'megapascal': 'megapascal', 'MEGAPASCAL': 'megapascal',
    'bar': 'bar', 'BAR': 'bar', 'mbar': 'millibar', 'MBAR': 'millibar', 'millibar': 'millibar', 'MILLIBAR': 'millibar',
    'psi': 'pound per square inch', 'PSI': 'pound per square inch',

    # Energy Units
    'j': 'joule', 'J': 'joule', 'joule': 'joule', 'JOULE': 'joule',
    'kj': 'kilojoule', 'KJ': 'kilojoule', 'kilojoule': 'kilojoule', 'KILOJOULE': 'kilojoule',
    'cal': 'calorie', 'CAL': 'calorie', 'calorie': 'calorie', 'CALORIE': 'calorie',
    'kcal': 'kilocalorie', 'KCAL': 'kilocalorie', 'kilocalorie': 'kilocalorie', 'KILOCALORIE': 'kilocalorie',
    'wh': 'watt-hour', 'WH': 'watt-hour', 'watt-hour': 'watt-hour', 'WATT-HOUR': 'watt-hour',
    'kwh': 'kilowatt-hour', 'KWH': 'kilowatt-hour', 'kilowatt-hour': 'kilowatt-hour', 'KILOWATT-HOUR': 'kilowatt-hour',

    # Time Units
    's': 'second', 'S': 'second', 'sec': 'second', 'SEC': 'second', 'seconds': 'second', 'SECONDS': 'second',
    'min': 'minute', 'MIN': 'minute', 'minutes': 'minute', 'MINUTES': 'minute',
    'h': 'hour', 'H': 'hour', 'hr': 'hour', 'HR': 'hour', 'hours': 'hour', 'HOURS': 'hour',
    'day': 'day', 'DAY': 'day', 'days': 'day', 'DAYS': 'day',

    # Speed Units
    'm/s': 'meter per second', 'M/S': 'meter per second', 'meter per second': 'meter per second', 'METER PER SECOND': 'meter per second',
    'km/h': 'kilometer per hour', 'KM/H': 'kilometer per hour', 'kilometer per hour': 'kilometer per hour', 'KILOMETER PER HOUR': 'kilometer per hour',
    'mph': 'miles per hour', 'MPH': 'miles per hour', 'mile per hour': 'mile per hour', 'MILE PER HOUR': 'mile per hour',
    'knot': 'knot', 'KNOT': 'knot',

    # Area Units
    'sqm': 'square meter', 'SQM': 'square meter', 'square meter': 'square meter', 'SQUARE METER': 'square meter',
    'sqcm': 'square centimeter', 'SQCM': 'square centimeter', 'square centimeter': 'square centimeter', 'SQUARE CENTIMETER': 'square centimeter',
    'sqmm': 'square millimeter', 'SQMM': 'square millimeter', 'square millimeter': 'square millimeter', 'SQUARE MILLIMETER': 'square millimeter',
    'sqft': 'square foot', 'SQFT': 'square foot', 'square foot': 'square foot', 'SQUARE FOOT': 'square foot',
    'acre': 'acre', 'ACRE': 'acre',
    'hectare': 'hectare', 'HECTARE': 'hectare',
}



# Function to normalize the unit
def normalize_units(unit_value):
    # Convert to lowercase and strip any spaces
    unit_value = unit_value.lower().strip()

    # Replace abbreviations with full unit names using regex
    for abbr, full_unit in unit_mapping.items():
        unit_value = re.sub(rf'\b{abbr}\b', full_unit, unit_value)

    return unit_value

# Function to compare entity_value with extracted_units
def compare_units(extracted_units, entity_value):
    # Handle None or empty values
    if pd.isna(extracted_units) or pd.isna(entity_value):
        return False

    # Normalize and clean up spacing and case
    extracted_units = normalize_units(extracted_units)
    entity_value = normalize_units(entity_value)

    # If extracted_units is a list or multiple values, check if entity_value is in that list
    if isinstance(extracted_units, str):
        extracted_units_list = [unit.strip() for unit in extracted_units.split(",")]
        return any(entity_value in unit for unit in extracted_units_list)

    # Default case, simple comparison
    return extracted_units == entity_value

# Apply the comparison function to each row
dummy_train['comparison_result'] = dummy_train.apply(
    lambda row: compare_units(row['extracted_units'], row['entity_value']), axis=1
)

# Check comparison results
print(dummy_train[['image_link', 'extracted_units', 'entity_value', 'comparison_result']].head())

# Filter rows where extracted_units is not None for CNN training
filtered_dummy_train = dummy_train[dummy_train['extracted_units'].notna()]



                                          image_link  \
0  https://m.media-amazon.com/images/I/81nEfmLyyM...   
1  https://m.media-amazon.com/images/I/51gz4v5BxT...   
2  https://m.media-amazon.com/images/I/61D2A-LxNn...   
3  https://m.media-amazon.com/images/I/51INxCAF0T...   
4  https://m.media-amazon.com/images/I/81-zmM+W8B...   

                          extracted_units  entity_value  comparison_result  
0  1 KG, 100 g, 50 mg, 27 mg, 105 g, 64 g  1.0 kilogram              False  
1                                    None     78.0 inch              False  
2                                    None     10.1 inch              False  
3                                    None     2.88 inch              False  
4                                  1.8 lb     1.8 pound               True  


In [ ]:
dummy_train['extracted_text'][2]

'«\n\nx'

In [ ]:
# Count how many true values are in the 'comparison_result' field
true_count = filtered_dummy_train['comparison_result'].sum()

print(f"Number of True values in 'comparison_result': {true_count/1000}")
print(filtered_dummy_train.count())
print(true_count)


Number of True values in 'comparison_result': 0.047
image_link           163
group_id             163
entity_name          163
entity_value         163
extracted_text       163
extracted_units      163
comparison_result    163
dtype: int64
47


In [ ]:
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import img_to_array

# Function to preprocess image
def preprocess_image(image_url):
    try:
        response = requests.get(image_url)
        img = Image.open(BytesIO(response.content))
        img = img.resize((128, 128))  # Resize to match CNN input
        img = img_to_array(img)
        img = img / 255.0  # Normalize pixel values
        return img
    except Exception as e:
        print(f"Error loading image {image_url}: {e}")
        return None

# Preprocess all images in the filtered training data
filtered_dummy_train['image_data'] = filtered_dummy_train['image_link'].apply(preprocess_image)

# Remove rows with None images
filtered_dummy_train = filtered_dummy_train[filtered_dummy_train['image_data'].notna()]

# Convert the image data to numpy arrays
X_train = np.stack(filtered_dummy_train['image_data'].values)
y_train = pd.get_dummies(filtered_dummy_train['entity_value']).values  # One-hot encode the entity values

<ipython-input-126-99442c5e2d9e>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dummy_train['image_data'] = filtered_dummy_train['image_link'].apply(preprocess_image)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define CNN model
def create_cnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))  # Add dropout for regularization
    model.add(Dense(num_classes, activation='softmax'))  # Output layer for classification

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Get input shape and number of classes
input_shape = X_train[0].shape
num_classes = y_train.shape[1]

# Create and compile the model
cnn_model = create_cnn_model(input_shape, num_classes)
cnn_model.summary()  # View model structure

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_15 (Conv2D)                   │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_15 (MaxPooling2D)      │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_16 (Conv2D)                   │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_16 (MaxPooling2D)      │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_17 (Conv2D)                   │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_17 (MaxPooling2D)      │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_5 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 262)                 │          33,798 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,338,438 (12.74 MB)

 Trainable params: 3,338,438 (12.74 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Split the data into training and validation sets
from sklearn.model_selection import train_test_split

X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Train the model
cnn_model.fit(X_train_split, y_train_split, validation_data=(X_val_split, y_val_split), epochs=10, batch_size=32)

# Save the trained model
cnn_model.save('entity_value_cnn_model.h5')

Epoch 1/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.0069 - loss: 5.6699 - val_accuracy: 0.0000e+00 - val_loss: 5.5701
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.0106 - loss: 5.5512 - val_accuracy: 0.0000e+00 - val_loss: 5.6401
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.0142 - loss: 5.4772 - val_accuracy: 0.0000e+00 - val_loss: 5.6179
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.0095 - loss: 5.4803 - val_accuracy: 0.0000e+00 - val_loss: 5.8079
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.0133 - loss: 5.3861 - val_accuracy: 0.0000e+00 - val_loss: 6.0828
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.0230 - loss: 5.2112 - val_accuracy: 0.0000e+00 - val_loss: 6.2089
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.0553 - loss: 5.0256 - val_accuracy: 0.0000e+00 - val_loss: 6.3680
Epoch 8/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.0790 - loss: 4.7580 - val_accuracy: 0.0000

In [ ]:
filtered_dummy_train.head()

,image_link,group_id,entity_name,entity_value,extracted_text,extracted_units,comparison_result,image_data
1,https://m.media-amazon.com/images/I/61LLE5kWCJ...,630869,item_weight,20.0 gram,Malachite\n\nRhodium Plated\n\n \n\nSeries: Br...,20.0 g,True,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,..."
2,https://m.media-amazon.com/images/I/81gceM5l2i...,281678,item_weight,200 gram,CONTENT: Natural Green Tea\nINGREDIENTS: Green...,"2 g, 200 gm",True,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,..."
6,https://m.media-amazon.com/images/I/61EOEwhxPd...,107694,item_weight,"[10.0, 14.0] kilogram",SIZE SELECTI\nAge Height\n\n2-3 years —-85-10...,"14 kg, 32 lb, 20 kg, 44 lb, 27 kg, 59 lb",False,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,..."
8,https://m.media-amazon.com/images/I/71YOloPzwq...,523149,item_weight,23.0 gram,HEIGHT:21MM\nWIOTH:28MM\nWEIGHT:25G,25 G,False,"[[[0.9882353, 0.9882353, 0.9882353], [0.988235..."
10,https://m.media-amazon.com/images/I/71S5VcBG45...,844474,item_weight,1.3 kilogram,Product information\n\n{Brand] | FANDARE\n{Mat...,1.3 kg,True,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,..."


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score

# Load the trained model
cnn_model = load_model('entity_value_cnn_model.h5')

# Assuming the images have already been preprocessed as X_test

# Create a label encoder to convert y_test (strings) to numeric labels
label_encoder = LabelEncoder()

# Fit the label encoder on the true labels (y_test)
y_test_encoded = label_encoder.fit_transform(y_test)

# Make predictions using the loaded model
y_pred = cnn_model.predict(X_test)

# Convert the model's predictions to class indices
y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate precision, recall, and F1 score using numeric labels
precision = precision_score(y_test_encoded, y_pred_classes, average='weighted')
recall = recall_score(y_test_encoded, y_pred_classes, average='weighted')
f1 = f1_score(y_test_encoded, y_pred_classes, average='weighted')

# Print the results
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 217ms/step
Precision: 0.0
Recall: 0.0
F1 Score: 0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
